In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.types import *

import pyspark.sql.functions as F

spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [2]:
# Read lines from the text file

answers_sdf = spark.read.load('sx-stackoverflow-a2q.txt', format="text")
comments_answers_sdf = spark.read.load('sx-stackoverflow-c2a.txt', format="text")
comments_questions_sdf = spark.read.load('sx-stackoverflow-c2q.txt', format="text")

## Step 2.2

In [3]:
answers_sdf.count()

17823525

In [4]:
answers_sdf.show(10)

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
+----------------+
only showing top 10 rows



In [5]:
answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
comments_answers_sdf.count()

25405374

In [7]:
comments_answers_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
+--------------------+
only showing top 10 rows



In [8]:
comments_answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
comments_questions_sdf.count()

20268151

In [10]:
comments_questions_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|4550 4550 1220729190|
|  242 184 1220733503|
|4213 4946 1220768149|
|    91 91 1220768295|
|2658 1874 1220771891|
|4035 1874 1220773037|
|2257 4489 1220802041|
|  577 577 1220834891|
|4489 4489 1220853536|
| 828 2783 1220854143|
+--------------------+
only showing top 10 rows



In [11]:
comments_questions_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
answers_sdf.createOrReplaceTempView('answers_view')
answers_sdf = spark.sql('''SELECT CAST(split(value, " ")[0] AS integer) AS from_node,
                                  CAST(split(value, " ")[1] AS integer) AS to_node,
                                  split(value, " ")[2] AS timestamp,
                                  "answer" AS edge_type FROM answers_view''')
answers_sdf = answers_sdf.select('from_node','to_node', F.from_unixtime(answers_sdf.timestamp, "YYYY:MM:dd hh:mm:ss").alias("timestamp"), 'edge_type')

In [13]:
comments_answers_sdf.createOrReplaceTempView('comments_answers_view')
comments_answers_sdf = spark.sql('''SELECT CAST(split(value, " ")[0] AS integer) AS from_node,
                                  CAST(split(value, " ")[1] AS integer) AS to_node,
                                  split(value, " ")[2] AS timestamp,
                                  "comment-on-answer" AS edge_type FROM comments_answers_view''')
comments_answers_sdf = comments_answers_sdf.select('from_node','to_node', F.from_unixtime(comments_answers_sdf.timestamp, "YYYY:MM:dd hh:mm:ss").alias("timestamp"), 'edge_type')

In [14]:
comments_questions_sdf.createOrReplaceTempView('comments_questions_view')
comments_questions_sdf = spark.sql('''SELECT CAST(split(value, " ")[0] AS integer) AS from_node,
                                  CAST(split(value, " ")[1] AS integer) AS to_node,
                                  split(value, " ")[2] AS timestamp,
                                  "comment-on-question" AS edge_type FROM comments_questions_view''')
comments_questions_sdf = comments_questions_sdf.select('from_node','to_node', F.from_unixtime(comments_questions_sdf.timestamp, "YYYY:MM:dd hh:mm:ss").alias("timestamp"), 'edge_type')

In [15]:
graph_sdf = answers_sdf.unionAll(comments_questions_sdf).unionAll(comments_answers_sdf)

## Step 2.3

In [16]:
answers_sdf.count()

17823525

In [17]:
answers_sdf.show(5)

+---------+-------+-------------------+---------+
|from_node|to_node|          timestamp|edge_type|
+---------+-------+-------------------+---------+
|        9|      8|2008:08:01 05:17:57|   answer|
|        1|      1|2008:08:01 06:56:41|   answer|
|       13|      1|2008:08:01 03:57:27|   answer|
|       17|      1|2008:08:01 07:07:19|   answer|
|       48|      2|2008:08:01 07:16:22|   answer|
+---------+-------+-------------------+---------+
only showing top 5 rows



In [18]:
answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [19]:
comments_answers_sdf.count()

25405374

In [20]:
comments_answers_sdf.show(5)

+---------+-------+-------------------+-----------------+
|from_node|to_node|          timestamp|        edge_type|
+---------+-------+-------------------+-----------------+
|        1|     91|2008:09:06 03:07:10|comment-on-answer|
|        3|     91|2008:09:06 03:09:52|comment-on-answer|
|      380|    350|2008:09:06 03:42:16|comment-on-answer|
|     4642|   2257|2008:09:06 08:51:47|comment-on-answer|
|     4642|1324220|2008:09:06 09:15:46|comment-on-answer|
+---------+-------+-------------------+-----------------+
only showing top 5 rows



In [21]:
comments_answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [22]:
comments_questions_sdf.count()

20268151

In [23]:
comments_questions_sdf.show(5)

+---------+-------+-------------------+-------------------+
|from_node|to_node|          timestamp|          edge_type|
+---------+-------+-------------------+-------------------+
|     4550|   4550|2008:09:06 07:26:30|comment-on-question|
|      242|    184|2008:09:06 08:38:23|comment-on-question|
|     4213|   4946|2008:09:07 06:15:49|comment-on-question|
|       91|     91|2008:09:07 06:18:15|comment-on-question|
|     2658|   1874|2008:09:07 07:18:11|comment-on-question|
+---------+-------+-------------------+-------------------+
only showing top 5 rows



In [24]:
comments_questions_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [25]:
graph_sdf.count()

63497050

In [26]:
graph_sdf.show(5)

+---------+-------+-------------------+---------+
|from_node|to_node|          timestamp|edge_type|
+---------+-------+-------------------+---------+
|        9|      8|2008:08:01 05:17:57|   answer|
|        1|      1|2008:08:01 06:56:41|   answer|
|       13|      1|2008:08:01 03:57:27|   answer|
|       17|      1|2008:08:01 07:07:19|   answer|
|       48|      2|2008:08:01 07:16:22|   answer|
+---------+-------+-------------------+---------+
only showing top 5 rows



In [27]:
graph_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [28]:
graph_sdf.createOrReplaceTempView('graph_view')
highest_indegree = spark.sql('''SELECT to_node AS node, count(from_node) AS indegree
                                FROM graph_view
                                GROUP BY to_node
                                ORDER BY count(from_node) desc
                                ''')
highest_outdegree = spark.sql('''SELECT from_node AS node, count(to_node) AS outdegree
                                FROM graph_view
                                GROUP BY from_node
                                ORDER BY count(to_node) desc
                                ''')

## Step 3

In [29]:
highest_indegree.show(5)

+-------+--------+
|   node|indegree|
+-------+--------+
|  22656|   93143|
| 548225|   54410|
|  29407|   45824|
| 157882|   39576|
|1144035|   38909|
+-------+--------+
only showing top 5 rows



In [30]:
highest_outdegree.show(5)

+-------+---------+
|   node|outdegree|
+-------+---------+
|  22656|   101663|
| 505088|    65522|
|  17034|    65260|
|1491895|    62746|
| 100297|    54330|
+-------+---------+
only showing top 5 rows



In [31]:
def spark_bfs(G, origins, max_depth):
    schema = StructType([StructField('node', IntegerType(), True)])
    result = spark.createDataFrame(origins, schema).withColumn('depth', F.lit(0))
    
    G.createOrReplaceTempView('graph')
    visited_sdf = spark.createDataFrame(origins, schema)
    visited_sdf.createOrReplaceTempView('visited')
    
    frontier_dup_sdf= spark.sql('''SELECT g.to_node AS node
                                FROM visited v INNER JOIN graph g
                                ON g.from_node = v.node
                             ''').cache()
    
    frontier_dup_sdf = frontier_dup_sdf.join(visited_sdf, frontier_dup_sdf.node == visited_sdf.node, 'leftanti')
    
    result = result.unionAll(frontier_dup_sdf.withColumn('depth', F.lit(1)))
    frontier_sdf = frontier_dup_sdf.distinct()
    visited_sdf = visited_sdf.unionAll(frontier_sdf)
    
    for i in range(2, max_depth + 1):
        frontier_sdf.createOrReplaceTempView('frontier')
        frontier_dup_sdf = spark.sql('''SELECT g.to_node AS node
                                 FROM frontier f INNER JOIN graph g
                                 ON g.from_node = f.node''').cache()
        frontier_dup_sdf = frontier_dup_sdf.join(visited_sdf, frontier_dup_sdf.node == visited_sdf.node, 'leftanti')
        result = result.unionAll(frontier_dup_sdf.withColumn('depth', F.lit(i)))
        frontier_sdf = frontier_dup_sdf.distinct()
        visited_sdf = visited_sdf.unionAll(frontier_sdf)
    return result

## Step 4.1

In [32]:
bfs_sdf = spark_bfs(comments_questions_sdf, [{'node': 4550}, {'node': 242}], 2)

In [33]:
bfs_sdf.count()

396

In [34]:
bfs_sdf.show()

+-------+-----+
|   node|depth|
+-------+-----+
|   4550|    0|
|    242|    0|
|1619254|    1|
|2332659|    1|
|5504881|    1|
|1139389|    1|
|1940564|    1|
| 818089|    1|
|3047450|    1|
|4773326|    1|
|5070920|    1|
|2495819|    1|
|   1551|    1|
| 602291|    1|
| 451383|    1|
|4569974|    1|
| 633513|    1|
|2705465|    1|
|2605058|    1|
|  67719|    1|
+-------+-----+
only showing top 20 rows



In [35]:
bfs_sdf[bfs_sdf['depth'] == 2].count()

366

In [36]:
filtered_bfs_sdf = bfs_sdf[bfs_sdf['depth'] == 2]
filtered_bfs_sdf.createOrReplaceTempView('subgraph')
filtered_bfs_sdf = spark.sql('''SELECT node, count(*)
                                FROM subgraph 
                                GROUP BY node
                                HAVING count(*) > 1''')
filtered_bfs_sdf.count()

42

In [ ]:
def friend_rec(filtered_bfs_sdf, graph_sdf):
    filtered_bfs_sdf.createOrReplaceTempView('duplicate')
    
    duplicate_copy_sdf = spark.sql('SELECT * FROM duplicate')
    duplicate_copy_sdf.createOrReplaceTempView('copy')
    product_sdf = spark.sql('SELECT duplicate.node AS from_node, copy.node AS to_node \
                             FROM duplicate CROSS JOIN copy').cache()
    product_sdf = product_sdf[product_sdf['from_node'] != product_sdf['to_node']]
    product_sdf.createOrReplaceTempView('product')
    print(product_sdf.count())
    
    graph_sdf.createOrReplaceTempView('graph')
    product_sdf = spark.sql('SELECT p.from_node, p.to_node FROM product p \
                            EXCEPT SELECT g.from_node, g.to_node FROM graph g')
    print(product_sdf.count())
    product_sdf.createOrReplaceTempView('product')
    product_sdf = spark.sql('SELECT p.to_node AS from_node, p.from_node AS to_node FROM product p \
                            EXCEPT SELECT g.from_node AS from_node, g.to_node AS to_node FROM graph g')
    print(product_sdf.count())
    return product_sdf


result = friend_rec(filtered_bfs_sdf, comments_questions_sdf)

1722
1721


## Step 4.2

In [55]:
result.count()

1721

In [39]:
result.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|    21918| 453447|
|    21918|4204628|
|    46646| 267679|
|    59017|1187554|
|   104015|  55503|
|   211452|  59017|
|   503032|2269511|
|  1406214| 503032|
|  2269511|  21918|
|  3830157| 132640|
|  4204628|  47550|
|    46646|1127460|
|    55503| 104015|
|    60602|1187554|
|   267679|1061543|
|   393268| 318811|
|  1218595|1187554|
|  1218595|4800193|
|    27483|  46646|
|    63775|  46646|
+---------+-------+
only showing top 20 rows

